# Install requirements

# Import libraries

In [1]:
from textblob import TextBlob
import pandas as pd
import numpy as np
import re,string
import chart_studio
import chart_studio.plotly as py
import plotly
import plotly.graph_objs as go
from plotly.offline import iplot
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl', offline=True)

# Read twitter data from the file in the working directory

In [2]:
#contains most trendy real time tweets 
#Duration: 14-06-2020 to 17-06-2020
#No of tweets: 20847
df=pd.read_csv("D:\Winter2020\covid1.csv") 
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8528 entries, 0 to 8527
Data columns (total 7 columns):
User         8528 non-null object
Text         8528 non-null object
Date         8528 non-null object
Retweets     8528 non-null int64
Favorites    8528 non-null int64
HashTags     8521 non-null object
City         8528 non-null object
dtypes: int64(2), object(5)
memory usage: 466.5+ KB


,User,Text,Date,Retweets,Favorites,HashTags,City
0,bimaldas1965,Pandemic is the result of loss of natural boun...,2020-04-13 22:56:36+00:00,0,1,#pandemic #covid19 #wildlife,Mumbai
1,freddzo,@AxisBank http://replied.Next they will ask me...,2020-04-13 22:27:59+00:00,0,0,#pathetic #bigticket #Help #COVID19,Mumbai
2,Andy_gawai,No schedule for sleeping/getting up/having foo...,2020-04-13 20:03:55+00:00,1,1,#IndiaFightsCorona #Covid19India #Covid_19 #CO...,Mumbai
3,AkshayMahapadi,@PiyushGoyal ji please declare d amendnent in ...,2020-04-13 20:02:01+00:00,2,2,#Ganpati #lockdown #COVID19 #Konkan #Goa #Karn...,Mumbai
4,him_paliwal,"Have we got vaccine for #Covid19? If not, then...",2020-04-13 18:49:53+00:00,2,2,#Covid19 #PleaseHelp #NeedtoSleep #CrackersKoS...,Mumbai


# Cleaning 

In [3]:
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())


# Sentiment Analyzer Function

In [4]:
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity ==0:
        return 'Neutral'
    else:
        return 'Negative'

# Creating New Columns

In [5]:
df['clean_tweet'] = df['Text'].apply(lambda x: clean_tweet(x))
df['Sentiment'] = df['clean_tweet'].apply(lambda x: analyze_sentiment(x))

# Testing for some tweets and their sentiments

In [6]:
n=245
print('Original tweet:\n'+ df['Text'][n])
print()
print('Clean tweet:\n'+df['clean_tweet'][n])
print()
print('Sentiment:\n'+df['Sentiment'][n])

Original tweet:
Everyone’s going live but nobody’s living. #COVID19

Clean tweet:
Everyone s going live but nobody s living COVID19

Sentiment:
Positive


In [7]:
n=2345
print('Original tweet:\n'+ df['Text'][n])
print()
print('Clean tweet:\n'+df['clean_tweet'][n])
print()
print('Sentiment:\n'+df['Sentiment'][n])

Original tweet:
#CoronaUpdatesInIndia See how systematically Private labs are taking for #Covid19 #WorldHealthDay2020 #Coronavirustruth #CoronavirusOutbreak #IndiaFightsCoronavirus

Clean tweet:
CoronaUpdatesInIndia See how systematically Private labs are taking for Covid19 WorldHealthDay2020 Coronavirustruth CoronavirusOutbreak IndiaFightsCoronavirus

Sentiment:
Neutral


In [8]:
n=4
print('Original tweet:\n'+ df['Text'][n])
print()
print('Clean tweet:\n'+df['clean_tweet'][n])
print()
print('Sentiment:\n'+df['Sentiment'][n])

Original tweet:
Have we got vaccine for #Covid19? If not, then why are these people bursting crackers/rockets from past 1 hr? Location: Sai Nath Nagar, Wadgaon Sheri. #PleaseHelp #NeedtoSleep #CrackersKoStopKarona #KuchKarona @PuneCityPolice @CPPuneCity @DGPMaharashtra @SidShirole

Clean tweet:
Have we got vaccine for Covid19 If not then why are these people bursting crackers rockets from past 1 hr Location Sai Nath Nagar Wadgaon Sheri PleaseHelp NeedtoSleep CrackersKoStopKarona KuchKarona

Sentiment:
Negative


In [9]:
df.head()

,User,Text,Date,Retweets,Favorites,HashTags,City,clean_tweet,Sentiment
0,bimaldas1965,Pandemic is the result of loss of natural boun...,2020-04-13 22:56:36+00:00,0,1,#pandemic #covid19 #wildlife,Mumbai,Pandemic is the result of loss of natural boun...,Positive
1,freddzo,@AxisBank http://replied.Next they will ask me...,2020-04-13 22:27:59+00:00,0,0,#pathetic #bigticket #Help #COVID19,Mumbai,they will ask me to visit the nearest branch a...,Negative
2,Andy_gawai,No schedule for sleeping/getting up/having foo...,2020-04-13 20:03:55+00:00,1,1,#IndiaFightsCorona #Covid19India #Covid_19 #CO...,Mumbai,No schedule for sleeping getting up having foo...,Positive
3,AkshayMahapadi,@PiyushGoyal ji please declare d amendnent in ...,2020-04-13 20:02:01+00:00,2,2,#Ganpati #lockdown #COVID19 #Konkan #Goa #Karn...,Mumbai,ji please declare d amendnent in ARP over to 3...,Negative
4,him_paliwal,"Have we got vaccine for #Covid19? If not, then...",2020-04-13 18:49:53+00:00,2,2,#Covid19 #PleaseHelp #NeedtoSleep #CrackersKoS...,Mumbai,Have we got vaccine for Covid19 If not then wh...,Negative


# Sentiment Distribution

Overall:

In [10]:
df['Sentiment'].value_counts().iplot(kind='bar', xTitle='Sentiment',
                                    yTitle='Count', title='Overall Sentiment Distribution')

Most Popular:

In [11]:
df_popular = df[df['Retweets'] >= 500]
df_popular['Sentiment'].value_counts().iplot(kind='bar', xTitle='Sentiment',
                                    yTitle='Count', title = 'Sentiment Distribution for <br> popular tweets (Above 500)')

Average Popularity:

In [12]:
df_average = df[(df['Retweets'] <= 2000) & (df['Retweets'] >=500)]
df_average['Sentiment'].value_counts().iplot(kind='bar', xTitle='Sentiment',
                                    yTitle='Count', title = ('Sentiment Distribution for <br> averagely popular tweets between 500 and 2000'))

Least Popular:

In [13]:
df_unpopular = df[df['Retweets'] <= 100]
df_unpopular['Sentiment'].value_counts().iplot(kind='bar', xTitle='Sentiment',
                                    yTitle='Count', title = ('Sentiment Distribution for <br> unpopular tweets'))


# Save the dataset along with the clean tweet and their respective sentiments 


In [14]:
df.to_csv('textblobcovid1.csv',index=False)


# Read sentiments

In [15]:
#contains most trendy real time tweets 
#Duration: 14-06-2020 to 17-06-2020
#No of tweets: 20847
df=pd.read_csv("textblobcovid1.csv") 
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8528 entries, 0 to 8527
Data columns (total 9 columns):
User           8528 non-null object
Text           8528 non-null object
Date           8528 non-null object
Retweets       8528 non-null int64
Favorites      8528 non-null int64
HashTags       8521 non-null object
City           8528 non-null object
clean_tweet    8528 non-null object
Sentiment      8528 non-null object
dtypes: int64(2), object(7)
memory usage: 599.8+ KB


,User,Text,Date,Retweets,Favorites,HashTags,City,clean_tweet,Sentiment
0,bimaldas1965,Pandemic is the result of loss of natural boun...,2020-04-13 22:56:36+00:00,0,1,#pandemic #covid19 #wildlife,Mumbai,Pandemic is the result of loss of natural boun...,Positive
1,freddzo,@AxisBank http://replied.Next they will ask me...,2020-04-13 22:27:59+00:00,0,0,#pathetic #bigticket #Help #COVID19,Mumbai,they will ask me to visit the nearest branch a...,Negative
2,Andy_gawai,No schedule for sleeping/getting up/having foo...,2020-04-13 20:03:55+00:00,1,1,#IndiaFightsCorona #Covid19India #Covid_19 #CO...,Mumbai,No schedule for sleeping getting up having foo...,Positive
3,AkshayMahapadi,@PiyushGoyal ji please declare d amendnent in ...,2020-04-13 20:02:01+00:00,2,2,#Ganpati #lockdown #COVID19 #Konkan #Goa #Karn...,Mumbai,ji please declare d amendnent in ARP over to 3...,Negative
4,him_paliwal,"Have we got vaccine for #Covid19? If not, then...",2020-04-13 18:49:53+00:00,2,2,#Covid19 #PleaseHelp #NeedtoSleep #CrackersKoS...,Mumbai,Have we got vaccine for Covid19 If not then wh...,Negative


# No. Of Retweets vs. No. Of Fav Counts

In [16]:
df.iplot(x='Retweets', y = 'Favorites', mode='markers'
        , categories='Sentiment',layout=dict(
        xaxis=dict(type='log', title='No. Of Retweets'),
        yaxis=dict(type='log', title='No. Of Fav Counts'),
        title='No. Of Retweets vs. No. Of Fav Counts'))